# User Activity Forecasting - Optuna Hyperparameter Optimization

**Objective:** Predict how many days a user will be active in the next 7 days (0-7)

**Key Features:**
- Advanced feature engineering
- Optuna for Bayesian hyperparameter optimization
- Multiple model comparison (LightGBM, XGBoost, GradientBoosting)
- SHAP explainability
- Snowflake Model Registry integration

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, KFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold, SelectFromModel

import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

import shap

from snowflake.snowpark.context import get_active_session
from snowflake.ml.registry import Registry
from snowflake.ml.feature_store import FeatureStore
from snowflake.ml.experiment import ExperimentTracking

session = get_active_session()
registry = Registry(session=session, database_name="dev", schema_name="data_science")
exp = ExperimentTracking(session=session)
exp.set_experiment("User_activity_forecasting_models")

print("Libraries loaded successfully")

## 1. Data Loading from Feature Store

In [13]:
fs = FeatureStore(
    session=session,
    database="dev",
    name="user_activity_feature_store",
    default_warehouse="analytics_wh_flex"
)

fv_ads = fs.get_feature_view(name="user_features_ads", version="1.2")
fv_call_quality = fs.get_feature_view(name="user_features_call_quality", version="1.2")
fv_call_rating = fs.get_feature_view(name="user_features_call_rating", version="1.2")
fv_data_usage = fs.get_feature_view(name="user_features_data_usage", version="1.2")
fv_drawer_event = fs.get_feature_view(name="user_features_drawer_event", version="1.2")
fv_inbound_calls = fs.get_feature_view(name="user_features_inbound_calls", version="1.2")
fv_messages = fs.get_feature_view(name="user_features_messages", version="1.2")
fv_nps_rating = fs.get_feature_view(name="user_features_nps_rating", version="1.2")
fv_outbound_calls = fs.get_feature_view(name="user_features_outbound_calls", version="1.2")
fv_redial = fs.get_feature_view(name="user_features_redial", version="1.2")
fv_sessions = fs.get_feature_view(name="user_features_sessions", version="1.2")

all_features = [
    fv_ads, fv_call_quality, fv_call_rating, fv_data_usage,
    fv_drawer_event, fv_inbound_calls, fv_messages, fv_nps_rating,
    fv_outbound_calls, fv_redial, fv_sessions
]

print(f"Loaded {len(all_features)} feature views")

In [14]:
# spine df
spine_df = session.sql("""
WITH daily AS (
    SELECT username, date_utc, SUM(time_in_app_mins_per_day) AS mins
    FROM metrics_daily_userlevel_app_time_sessions
    WHERE date_utc >= DATEADD('day', -37, '2025-11-19')
    GROUP BY username, date_utc
),
labeled AS (
    SELECT up.user_id_hex, d.date_utc,
        COALESCE(SUM(IFF(d.mins > 1, 1, 0)) OVER (
            PARTITION BY d.username ORDER BY d.date_utc
            RANGE BETWEEN INTERVAL '1 DAY' FOLLOWING AND INTERVAL '7 DAYS' FOLLOWING
        ), 0) AS active_days_in_week
    FROM daily d
    JOIN user_profiles up ON d.username = up.latest_username
),
filtered AS (
    SELECT * FROM labeled
    WHERE date_utc <= DATEADD('day', -7, '2025-11-19')
),
sampled AS (
    SELECT *, ROW_NUMBER() OVER (
        PARTITION BY date_utc 
        ORDER BY HASH(user_id_hex || date_utc)
    ) AS rn
    FROM filtered
)
SELECT user_id_hex, date_utc, active_days_in_week 
FROM sampled 
WHERE rn <= 70000
""")

df = fs.generate_training_set(
    spine_df=spine_df,
    features=all_features,
    spine_label_cols="active_days_in_week"
)

df = df.to_pandas().fillna(0)
print(f"Dataset shape: {df.shape}")
print(f"Target distribution:\n{df['ACTIVE_DAYS_IN_WEEK'].value_counts().sort_index()}")

In [ ]:
df.head()

## 2. Exploratory Data Analysis

In [15]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['ACTIVE_DAYS_IN_WEEK'], bins=8, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Active Days in Next 7 Days')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Target Distribution')
axes[0].set_xticks(range(8))

target_counts = df['ACTIVE_DAYS_IN_WEEK'].value_counts().sort_index()
axes[1].bar(target_counts.index, target_counts.values / len(df) * 100, color='coral', edgecolor='black')
axes[1].set_xlabel('Active Days')
axes[1].set_ylabel('Percentage (%)')
axes[1].set_title('Target Distribution (Percentage)')
axes[1].set_xticks(range(8))

plt.tight_layout()
plt.show()

print(f"\nTarget Statistics:")
print(f"Mean: {df['ACTIVE_DAYS_IN_WEEK'].mean():.2f}")
print(f"Median: {df['ACTIVE_DAYS_IN_WEEK'].median():.2f}")
print(f"Std: {df['ACTIVE_DAYS_IN_WEEK'].std():.2f}")

In [16]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_with_target = df[numeric_cols].corr()['ACTIVE_DAYS_IN_WEEK'].abs().sort_values(ascending=False)

print("Top 20 Features Correlated with Target:")
print("=" * 50)
for feat, corr in corr_with_target.head(21).items():
    if feat != 'ACTIVE_DAYS_IN_WEEK':
        print(f"{feat}: {corr:.4f}")

top_features = corr_with_target.head(16).index.tolist()
top_corr = df[top_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(top_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, annot_kws={'size': 8})
plt.title('Top 15 Features - Correlation Matrix')
plt.tight_layout()
plt.show()

## 3. Advanced Feature Engineering

In [ ]:
def create_features(data):
    df_fe = data.copy()
    
    initial_rows = len(df_fe)
    df_fe = df_fe.drop_duplicates()
    duplicates_removed = initial_rows - len(df_fe)
    print(f'Removed {duplicates_removed} duplicate rows')
    
    numeric_cols = df_fe.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [c for c in numeric_cols if c not in ['USER_ID_HEX', 'DATE_UTC', 'ACTIVE_DAYS_IN_WEEK']]
    
    skewness = df_fe[numeric_cols].skew()
    skewed_cols = skewness[abs(skewness) > 1.0].index.tolist()
    for col in skewed_cols:
        df_fe[col] = np.log1p(df_fe[col].clip(lower=0))
    print(f'Applied log1p to {len(skewed_cols)} skewed columns')
    
    for col in numeric_cols:
        Q1 = df_fe[col].quantile(0.25)
        Q3 = df_fe[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df_fe[col] = df_fe[col].clip(lower=lower, upper=upper)
    print(f'Clipped outliers using IQR method for {len(numeric_cols)} columns')
    
    return df_fe

df_engineered = create_features(df)
print(f'Final shape: {df_engineered.shape}')

## 4. Data Preparation

In [18]:
exclude_cols = ['USER_ID_HEX', 'DATE_UTC', 'ACTIVE_DAYS_IN_WEEK']
feature_cols = [c for c in df_engineered.columns if c not in exclude_cols]

X = df_engineered[feature_cols]
y = df_engineered['ACTIVE_DAYS_IN_WEEK']

variance_selector = VarianceThreshold(threshold=0.01)
variance_selector.fit(X)
selected_mask = variance_selector.get_support()
removed_features = X.columns[~selected_mask].tolist()
X = X.loc[:, selected_mask]

print(f"Features after variance threshold: {X.shape[1]}")
print(f"Removed {len(removed_features)} low variance features")

# Temporal split: 80% of days for training, 20% for testing
dates = df_engineered['DATE_UTC']
unique_dates = sorted(dates.unique())
n_train_days = int(len(unique_dates) * 0.8)
cutoff_date = unique_dates[n_train_days - 1]

train_mask = dates <= cutoff_date
test_mask = dates > cutoff_date

X_train = X[train_mask].reset_index(drop=True)
X_test = X[test_mask].reset_index(drop=True)
y_train = y[train_mask].reset_index(drop=True)
y_test = y[test_mask].reset_index(drop=True)

print(f"\nTrain: {len(X_train):,} rows ({n_train_days} days, <= {cutoff_date})")
print(f"Test:  {len(X_test):,} rows ({len(unique_dates) - n_train_days} days, > {cutoff_date})")
print(f"Features: {X_train.shape[1]}")

feature_names = X.columns.tolist()

## 5. Optuna Hyperparameter Optimization

In [19]:
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'verbosity': -1,
        'n_jobs': -1
    }
    
    model = LGBMRegressor(**params)
    
    tscv = TimeSeriesSplit(n_splits=3)
    scores = cross_val_score(model, X_train, y_train, cv=tscv, 
                             scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    return -scores.mean()

print("Optimizing LightGBM...")
sampler = TPESampler(seed=42)
study_lgbm = optuna.create_study(direction='minimize', sampler=sampler)
study_lgbm.optimize(objective_lgbm, n_trials=5, show_progress_bar=True)

print(f"\nBest LightGBM RMSE: {study_lgbm.best_value:.4f}")
print(f"Best parameters: {study_lgbm.best_params}")

In [20]:
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'verbosity': 0,
        'n_jobs': -1
    }
    
    model = XGBRegressor(**params)
    
    tscv = TimeSeriesSplit(n_splits=3)
    scores = cross_val_score(model, X_train, y_train, cv=tscv,
                             scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    return -scores.mean()

print("Optimizing XGBoost...")
study_xgb = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study_xgb.optimize(objective_xgb, n_trials=10, show_progress_bar=True)

print(f"\nBest XGBoost RMSE: {study_xgb.best_value:.4f}")
print(f"Best parameters: {study_xgb.best_params}")

In [22]:
def objective_gb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'random_state': 42
    }
    
    model = GradientBoostingRegressor(**params)
    
    tscv = TimeSeriesSplit(n_splits=3)
    scores = cross_val_score(model, X_train, y_train, cv=tscv,
                             scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    return -scores.mean()

print("Optimizing GradientBoosting...")
study_gb = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study_gb.optimize(objective_gb, n_trials=5, show_progress_bar=True)

print(f"\nBest GradientBoosting RMSE: {study_gb.best_value:.4f}")
print(f"Best parameters: {study_gb.best_params}")

In [ ]:
results_comparison = pd.DataFrame({
    'Model': ['LightGBM', 'XGBoost', 'GradientBoosting'],
    'Best_CV_RMSE': [
        study_lgbm.best_value,
        study_xgb.best_value,
        study_gb.best_value
    ]
}).sort_values('Best_CV_RMSE')

print("Model Comparison (Cross-Validation RMSE):")
print("=" * 50)
print(results_comparison.to_string(index=False))

best_model_name = results_comparison.iloc[0]['Model']
print(f"\nBest Model: {best_model_name}")

fig, ax = plt.subplots(figsize=(8, 5))
colors = ['green' if m == best_model_name else 'steelblue' for m in results_comparison['Model']]
ax.barh(results_comparison['Model'], results_comparison['Best_CV_RMSE'], color=colors)
ax.set_xlabel('CV RMSE (lower is better)')
ax.set_title('Model Comparison - Optuna Optimization')
for i, v in enumerate(results_comparison['Best_CV_RMSE']):
    ax.text(v + 0.01, i, f'{v:.4f}', va='center')
plt.tight_layout()
plt.show()

## 6. Train Final Model with Best Parameters

In [ ]:
best_studies = {
    'LightGBM': (study_lgbm, LGBMRegressor),
    'XGBoost': (study_xgb, XGBRegressor),
    'GradientBoosting': (study_gb, GradientBoostingRegressor)
}

best_study, ModelClass = best_studies[best_model_name]
best_params = best_study.best_params.copy()

if best_model_name == 'LightGBM':
    best_params.update({'random_state': 42, 'verbosity': -1, 'n_jobs': -1})
elif best_model_name == 'XGBoost':
    best_params.update({'random_state': 42, 'verbosity': 0, 'n_jobs': -1})
else:
    best_params.update({'random_state': 42})

print(f"Training final {best_model_name} model...")
print(f"Parameters: {best_params}")

start_time = time.time()
final_model = ModelClass(**best_params)
final_model.fit(X_train, y_train)
training_time = time.time() - start_time

y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"\nTraining completed in {training_time:.2f} seconds")
print(f"\nTraining Metrics:")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  MAE:  {train_mae:.4f}")
print(f"  R²:   {train_r2:.4f}")
print(f"\nTest Metrics:")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  MAE:  {test_mae:.4f}")
print(f"  R²:   {test_r2:.4f}")

## 7. Model Evaluation & Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

axes[0, 0].scatter(y_test, y_pred_test, alpha=0.3, s=10)
axes[0, 0].plot([0, 7], [0, 7], 'r--', lw=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual Active Days')
axes[0, 0].set_ylabel('Predicted Active Days')
axes[0, 0].set_title('Actual vs Predicted')
axes[0, 0].legend()
axes[0, 0].set_xlim(-0.5, 7.5)
axes[0, 0].set_ylim(-0.5, 7.5)

residuals = y_test - y_pred_test
axes[0, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 1].axvline(x=0, color='red', linestyle='--', lw=2)
axes[0, 1].set_xlabel('Residual (Actual - Predicted)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title(f'Residual Distribution (Mean: {residuals.mean():.3f})')

axes[1, 0].scatter(y_pred_test, residuals, alpha=0.3, s=10)
axes[1, 0].axhline(y=0, color='red', linestyle='--', lw=2)
axes[1, 0].set_xlabel('Predicted Values')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].set_title('Residuals vs Predicted')

pred_rounded = np.clip(np.round(y_pred_test), 0, 7).astype(int)
actual_counts = pd.Series(y_test).value_counts().sort_index()
pred_counts = pd.Series(pred_rounded).value_counts().sort_index()

width = 0.35
x = np.arange(8)
axes[1, 1].bar(x - width/2, [actual_counts.get(i, 0) for i in range(8)], width, label='Actual', alpha=0.7)
axes[1, 1].bar(x + width/2, [pred_counts.get(i, 0) for i in range(8)], width, label='Predicted', alpha=0.7)
axes[1, 1].set_xlabel('Active Days')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Distribution Comparison')
axes[1, 1].legend()
axes[1, 1].set_xticks(x)

plt.tight_layout()
plt.show()

In [ ]:
if hasattr(final_model, 'feature_importances_'):
    importance = final_model.feature_importances_
elif hasattr(final_model, 'get_feature_importance'):
    importance = final_model.get_feature_importance()
else:
    importance = np.zeros(len(feature_names))

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
}).sort_values('importance', ascending=False)

print("Top 20 Most Important Features:")
print("=" * 60)
for i, row in importance_df.head(20).iterrows():
    print(f"{row['feature']}: {row['importance']:.4f}")

# plt.figure(figsize=(12, 10))
# top_n = 25
# top_features_df = importance_df.head(top_n)
# plt.barh(range(top_n), top_features_df['importance'].values, color='steelblue')
# plt.yticks(range(top_n), top_features_df['feature'].values)
# plt.xlabel('Importance')
# plt.title(f'Top {top_n} Feature Importances - {best_model_name}')
# plt.gca().invert_yaxis()
# plt.tight_layout()
# plt.show()

## 8. SHAP Explainability

In [ ]:
sample_size = min(1000, len(X_train))
X_sample = X_train.sample(n=sample_size, random_state=42)

print(f"Computing SHAP values for {sample_size} samples...")
explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_sample)

# Global SHAP importance = mean absolute SHAP
mean_abs_shap = np.mean(np.abs(shap_values), axis=0)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

plt.sca(axes[0])
shap.summary_plot(shap_values, X_sample, feature_names=feature_names, 
                  show=False, max_display=20)
axes[0].set_title('SHAP Feature Impact (Beeswarm)')

plt.sca(axes[1])
shap.summary_plot(shap_values, X_sample, feature_names=feature_names,
                  plot_type='bar', show=False, max_display=20)
axes[1].set_title('SHAP Feature Importance (Bar)')

plt.tight_layout()
plt.show()

## 9. Log to Snowflake Model Registry

In [ ]:
metrics_dict = {
    'train_rmse': float(train_rmse),
    'test_rmse': float(test_rmse),
    'train_mae': float(train_mae),
    'test_mae': float(test_mae),
    'train_r2': float(train_r2),
    'test_r2': float(test_r2),
    'training_time_seconds': float(training_time),
    'cv_rmse': float(best_study.best_value)
}

with exp.start_run():
    for metric_name, metric_value in metrics_dict.items():
        exp.log_metric(metric_name, metric_value)
    
    exp.log_param('model_type', best_model_name)
    exp.log_param('n_train_samples', len(X_train))
    exp.log_param('n_test_samples', len(X_test))
    exp.log_param('n_features', len(feature_names))
    exp.log_param('optuna_n_trials', 50)
    
    for param_name, param_value in best_params.items():
        if param_name not in ['random_state', 'verbosity', 'verbose', 'n_jobs']:
            exp.log_param(f'best_{param_name}', param_value)
    
    exp.log_model(
        model=final_model,
        model_name=f"{best_model_name}_best_model",
        metrics=metrics_dict,
        sample_input_data=X_train.head()
    )

print("Experiment logged successfully")

## 10. Summary

In [ ]:
print("="*70)
print("EXPERIMENT SUMMARY")
print("="*70)
print(f"\nObjective: Predict user active days in next 7 days (0-7)")
print(f"\nDataset:")
print(f"  - Total samples: {len(df):,}")
print(f"  - Training samples: {len(X_train):,}")
print(f"  - Test samples: {len(X_test):,}")
print(f"  - Original features: {df.shape[1]}")
print(f"  - Engineered features: {len(feature_names)}")

print(f"\nOptuna Optimization:")
print(f"  - Trials per model: 50")
print(f"  - Models compared: LightGBM, XGBoost, GradientBoosting")
print(f"  - Best model: {best_model_name}")
print(f"  - Best CV RMSE: {best_study.best_value:.4f}")

print(f"\nFinal Model Performance:")
print(f"  - Test RMSE: {test_rmse:.4f}")
print(f"  - Test MAE: {test_mae:.4f}")
print(f"  - Test R²: {test_r2:.4f}")
print(f"  - Training time: {training_time:.2f}s")

print(f"\nTop 5 Important Features:")
for i, row in importance_df.head(5).iterrows():
    print(f"  {i+1}. {row['feature']}: {row['importance']:.4f}")

print(f"\nModel registered")
print("="*70)

## 11. Select good features based on shap values, feature importance and remove correlated features and keep the one woth high shap value

In [ ]:
NUMERICAL_NOISE_EPS = 0.01
CORRELATION_THRESHOLD = 0.9

# Build a report table
feature_report = pd.DataFrame({
    "feature": feature_names,
    "importance": importance,
    "mean_abs_shap": mean_abs_shap
}).sort_values(["mean_abs_shap", "importance"], ascending=False)

print("Top features (SHAP then importance):")
print(feature_report.head(15))

# keep feature if either signal is non-zero
feature_report["keep"] = (
    (feature_report["importance"] > NUMERICAL_NOISE_EPS) |
    (feature_report["mean_abs_shap"] > NUMERICAL_NOISE_EPS)
)

selected_features = feature_report.loc[feature_report["keep"], "feature"].tolist()
dropped_features_zero_signal = feature_report.loc[~feature_report["keep"], "feature"].tolist()

print("\nSelected features (before correlation):", len(selected_features))
print("\nSelected features (before correlation) list:", selected_features)
print("Dropped features (zero signal):", len(dropped_features_zero_signal))
print("Dropped (zero signal) list:", dropped_features_zero_signal)

## Correlation pruning ONLY among selected features

# feature -> SHAP strength
shap_strength_by_feature = dict(
    zip(feature_report["feature"], feature_report["mean_abs_shap"])
)

# Correlation among selected features
correlation_matrix = X_train[selected_features].corr().abs()

# Find correlated feature pairs (upper triangle only)
row_idx, col_idx = np.where(
    np.triu(correlation_matrix.values, k=1) > CORRELATION_THRESHOLD
)

features_to_drop_due_to_correlation = set()
correlation_feature_names = correlation_matrix.columns


for i, j in zip(row_idx, col_idx):
    feature_a = correlation_feature_names[i]
    feature_b = correlation_feature_names[j]

    # Drop the weaker feature (smaller SHAP contribution)
    if shap_strength_by_feature[feature_a] >= shap_strength_by_feature[feature_b]:
        features_to_drop_due_to_correlation.add(feature_b)
    else:
        features_to_drop_due_to_correlation.add(feature_a)

final_selected_features = [
    f for f in selected_features if f not in features_to_drop_due_to_correlation
]

print("\nDropped due to correlation:", len(features_to_drop_due_to_correlation))
print("Dropped (correlation) list:", sorted(features_to_drop_due_to_correlation))
print("Final selected features:", len(final_selected_features))
print("Final feature list:", final_selected_features)

## Next Steps
The selected features from SHAP analysis can be used to update the production config for a more focused model.